In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

# df=pd.DataFrame()  # 初始化空值
# vTest = 1000


In [8]:
xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree
print( str(xl_dgree) )

sql = "SELECT * \
from day_zhibiao \
where \
jx_days_ud60 > 20 and jx_days_ud60 < 50 and \
jx_xl_60>-"+str(xl_dgree)+" and  \
jx_xl_20>0 "
print(sql)
# and \
#     lj_fl_3_10>1 and lj_fl_5_20>1 
df = pd.read_sql_query(sql, conn)
df.shape

# ds_qf60_1 ：jx_days_ud60 > 20 and jx_days_ud60 < 40 and jx_xl_60>-"+str(xl_02dgree)+" and jx_xl_10>0 and jx_xl_20>0 
# ds_qf60_2 ：jx_days_ud60 > 15 and jx_days_ud60 < 60 and jx_xl_60>-"+str(xl_02dgree)+" and jx_xl_10>0 and jx_xl_20>0 
# ds_qf60_3 ：jx_days_ud60 > 10 and jx_days_ud60 < 60 and jx_xl_60>-"+str(xl_02dgree)+"                and jx_xl_20>0 


0.00523
SELECT * from day_zhibiao where jx_days_ud60 > 20 and jx_days_ud60 < 50 and jx_xl_60>-0.00523 and  jx_xl_20>0 


(292325, 64)

In [9]:
# 检视数据状态
col_list = ['ts_code', 'trade_date', 'close', 'pre_close', 'pct_change', 'volume', \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_xl_250','jx_xl_120', 'jx_xl_60', \
    'jx_xl_20', 'jx_xl_10', 'jx_xl_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20', 'lj_fl_20_60', \
    'sz_max_5', 'sz_max_10', 'sz_max_20' ]
# col_list
df_qf60 = df[col_list].copy()
# df_qf60.info()

In [10]:
""" 数据抽查 """
df[ (df['sz_max_10']>10) ]


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_change,volume,...,lj_fl_5_60,lj_fl_20_60,sz_sum_5,sz_sum_10,sz_sum_20,sz_max_5,sz_max_10,sz_max_20,ema_fast,ema_slow
28,000001.SZ,20210903,18.50,18.50,17.70,18.04,18.40,-0.36,-1.9565,1394820.00,...,1.890600,1.271610,13.5697,3.3009,7.9113,14.024400,14.0244,14.0244,None,None
29,000001.SZ,20210902,18.00,18.78,17.80,18.40,17.88,0.52,2.9083,2422600.00,...,1.771710,1.245760,3.3500,4.2270,6.2125,4.565220,11.7935,11.7935,None,None
30,000001.SZ,20210827,18.56,18.88,18.31,18.39,18.46,-0.07,-0.3792,728820.00,...,1.097520,1.140190,-1.7906,11.7791,-1.4483,0.054377,11.8543,11.8543,None,None
42,000001.SZ,20190618,12.67,12.85,12.59,12.80,12.67,0.13,1.0260,483555.00,...,0.542438,0.741536,5.0027,11.5415,8.6084,7.812500,10.7812,10.7812,None,None
101,000002.SZ,20221124,16.03,16.17,15.76,16.02,15.49,0.53,3.4216,1316720.00,...,0.837376,1.225400,17.8693,20.1264,17.0654,18.601700,20.9738,27.7154,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292307,689009.SH,20210817,65.13,66.33,64.10,64.50,65.13,-0.63,-0.9673,7509.06,...,0.829043,0.794421,11.1517,17.8464,18.7235,11.209300,33.4574,33.4574,None,None
292308,689009.SH,20210816,64.00,65.93,63.12,65.13,63.80,1.33,2.0846,9409.06,...,0.827066,0.811494,1.5026,16.4237,20.3536,1.335790,32.1664,32.1664,None,None
292309,689009.SH,20210812,66.73,69.80,66.20,67.76,67.99,-0.23,-0.3383,18661.90,...,0.745676,0.817692,-3.8451,25.2919,25.6831,0.000000,27.0366,27.0366,None,None
292310,689009.SH,20210811,63.14,68.17,62.00,67.99,63.13,4.86,7.6984,23668.20,...,0.667078,0.810272,-3.3753,26.0921,27.0774,0.000000,26.6069,26.6069,None,None


In [11]:
""" 形成数据集 """

# 确定训练的主要目标s：
df['test5_8'] = df['sz_max_5'] > 8
df['test10_10'] = df['sz_max_10'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# 确定输出列  
col_list_ds = [ 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_xl_250','jx_xl_120', 'jx_xl_60', \
    'jx_xl_20', 'jx_xl_10', 'jx_xl_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20', 'lj_fl_20_60', \
    'test5_8', 'test10_10', 'test20_20', 'test20_30' ]
df_ds = df[col_list_ds].copy()
# df[ df['sz_max_10']>10 ].shape, df[ df['sz_max_20']>20 ].shape, df[ df['sz_max_20']>30 ].shape
df_ds[ df_ds['test5_8'] ].shape, df_ds[ df_ds['test10_10'] ].shape, df_ds[ df_ds['test20_20'] ].shape, df_ds[ df_ds['test20_30'] ].shape

((31862, 23), (45604, 23), (27473, 23), (10810, 23))

In [12]:
""" 输出到mysql """
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', conn, index=False, if_exists='append', chunksize=2000) 

292325